In [ ]:
!pip uninstall -y protobuf


Found existing installation: protobuf 4.25.5
Uninstalling protobuf-4.25.5:
  Successfully uninstalled protobuf-4.25.5


In [ ]:
!pip install protobuf==3.20.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 32.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.62.3 requires protobuf>=4.21.6, but you have protobuf 3.20.3 which is incompatible.


In [ ]:
!pip install -q groq langchain-groq litellm PyPDF2 chromadb text-generation langchain sentence-transformers langchain-community rank_bm25 gradio



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 86.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 14.9 MB/s eta 0:00

In [ ]:
print("Testing imports...")
import os
import gradio as gr
from typing import Dict, List, Optional
from google.colab import files, userdata

from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import Chroma
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain_groq import ChatGroq
print("All imports successful!")

Testing imports...
All imports successful!


In [ ]:
class AmazonReturnPolicyAssistant:
    def __init__(self, api_key: str):
        print("Initializing assistant...")
        os.environ["GROQ_API_KEY"] = api_key
        print("API key set")

        print("Setting up LLM...")
        self.llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0)
        print("LLM setup complete")

        print("Setting up embeddings...")
        self.embeddings = HuggingFaceEmbeddings(
            model_name="sentence-transformers/all-MiniLM-l6-v2",
            model_kwargs={'device': 'cpu'},
            encode_kwargs={'normalize_embeddings': False}
        )
        print("Embeddings setup complete")
        print("Initialization complete!")

    def process_document(self, file_path: str):
        print(f"Processing document: {file_path}")
        loader = TextLoader(file_path)
        data = loader.load()
        print("Document loaded")

        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=100,
            chunk_overlap=50
        )
        docs = text_splitter.split_documents(data)
        print("Document split into chunks")

        vectordb = Chroma.from_documents(docs, self.embeddings)
        vector_retriever = vectordb.as_retriever(
            search_type="mmr",
            search_kwargs={"k": 1}
        )
        print("Vector retriever created")

        bm25_retriever = BM25Retriever.from_documents(docs)
        bm25_retriever.k = 2
        print("BM25 retriever created")

        ensemble_retriever = EnsembleRetriever(
            retrievers=[bm25_retriever, vector_retriever],
            weights=[0.2, 0.8]
        )
        print("Ensemble retriever created")

        prompt = PromptTemplate(
            template="""You are an Amazon Return Policy Assistant, expert in providing clear and concise information.
            Context: {context}
            Question: {question}
            Please provide a clear and helpful response.""",
            input_variables=["context", "question"]
        )
        print("Prompt template created")

        qa_chain = RetrievalQA.from_chain_type(
            llm=self.llm,
            chain_type="stuff",
            retriever=ensemble_retriever,
            chain_type_kwargs={"prompt": prompt}
        )
        print("QA chain created")
        return qa_chain

In [ ]:
print("Testing file upload...")
uploaded = files.upload()
file_name = list(uploaded.keys())[0]
print(f"File uploaded: {file_name}")

print("Getting API key...")
groq_api_key = userdata.get('groq_api')
if not groq_api_key:
    raise ValueError("GROQ API key not found in Colab secrets")
print("API key retrieved")

Testing file upload...


Saving rag-evaluation.py to rag-evaluation.py
File uploaded: rag-evaluation.py
Getting API key...
API key retrieved


In [ ]:
print("Creating assistant...")
assistant = AmazonReturnPolicyAssistant(groq_api_key)
qa_chain = assistant.process_document(file_name)
print("Assistant ready!")

Creating assistant...
Initializing assistant...
API key set
Setting up LLM...
LLM setup complete
Setting up embeddings...
Embeddings setup complete
Initialization complete!
Processing document: rag-evaluation.py
Document loaded
Document split into chunks
Vector retriever created
BM25 retriever created
Ensemble retriever created
Prompt template created
QA chain created
Assistant ready!


In [ ]:
def create_interface(qa_chain):
    def query_handler(query: str) -> str:
        try:
            return qa_chain.run({"query": query})
        except Exception as e:
            return f"Error processing query: {str(e)}"

    interface = gr.Interface(
        fn=query_handler,
        inputs=gr.Textbox(
            lines=4,
            placeholder="Ask about Amazon's return policies...",
            label="Your Question"
        ),
        outputs=gr.Textbox(label="Answer"),
        title="Amazon Return Policy Assistant",
        examples=[["What is the general return policy?"]]
    )
    return interface

print("Launching interface...")
interface = create_interface(qa_chain)
interface.launch()




Launching interface...
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://88fba1e60301a2d390.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


RAG Evaluation

In [ ]:
!pip install numpy nltk rouge-score scikit-learn


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=11ea4e0c706fc1dafa5ed916a67843c52f644864fa6c10b17f77598602be9c8f
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score


In [ ]:
import numpy as np
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from typing import List, Dict, Optional
from statistics import mean

class RAGEvaluator:
    def __init__(self):
        """Initialize the RAG evaluator with required NLTK resources and scoring tools."""
        try:
            nltk.download('punkt', quiet=True)
        except:
            pass
        self.scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
        self.smoothie = SmoothingFunction().method1
        self.vectorizer = CountVectorizer(stop_words='english')

    def _compute_similarity(self, text1: str, text2: str) -> float:
        """
        Compute cosine similarity between two texts.

        Args:
            text1: First text string
            text2: Second text string

        Returns:
            float: Cosine similarity score
        """
        if not text1.strip() or not text2.strip():
            return 0.0

        try:
            vectors = self.vectorizer.fit_transform([text1, text2])
            return cosine_similarity(vectors[0:1], vectors[1:2])[0][0]
        except Exception:
            return 0.0

    def evaluate_single_response(self,
                               query: str,
                               retrieved_context: str,
                               generated_answer: str,
                               ground_truth: str) -> Dict[str, float]:
        """
        Evaluate a single RAG response using multiple metrics.

        Args:
            query: Original question or query
            retrieved_context: Retrieved context used to generate the answer
            generated_answer: Model-generated answer
            ground_truth: Ground truth (human) answer

        Returns:
            Dict containing evaluation metrics
        """
        # Ensure all inputs are strings
        query = str(query).strip()
        retrieved_context = str(retrieved_context).strip()
        generated_answer = str(generated_answer).strip()
        ground_truth = str(ground_truth).strip()

        # Context relevance score
        context_relevance = self._compute_similarity(query, retrieved_context)

        # Answer similarity metrics
        answer_similarity = self._compute_similarity(ground_truth, generated_answer)

        # BLEU score calculation
        try:
            ground_truth_tokens = nltk.word_tokenize(ground_truth.lower())
            generated_tokens = nltk.word_tokenize(generated_answer.lower())
            bleu_score = sentence_bleu([ground_truth_tokens], generated_tokens,
                                     smoothing_function=self.smoothie)
        except Exception:
            bleu_score = 0.0

        # ROUGE scores calculation
        try:
            rouge_scores = self.scorer.score(ground_truth, generated_answer)
            rouge1 = rouge_scores['rouge1'].fmeasure
            rouge2 = rouge_scores['rouge2'].fmeasure
            rougeL = rouge_scores['rougeL'].fmeasure
        except Exception:
            rouge1 = rouge2 = rougeL = 0.0

        # Faithfulness score (how well the answer aligns with the context)
        faithfulness_score = self._compute_similarity(retrieved_context, generated_answer)

        return {
            'context_relevance': context_relevance,
            'answer_similarity_cosine': answer_similarity,
            'answer_similarity_bleu': bleu_score,
            'answer_rouge1': rouge1,
            'answer_rouge2': rouge2,
            'answer_rougeL': rougeL,
            'faithfulness': faithfulness_score,
            'overall_score': mean([context_relevance, answer_similarity, bleu_score,
                                 rouge1, rouge2, rougeL, faithfulness_score])
        }

def evaluate_questions(questions: List[str],
                      human_answers: List[str],
                      predicted_answers: List[str],
                      retrieved_contexts: Optional[List[str]] = None) -> Dict[str, Dict[str, float]]:
    """
    Evaluate multiple questions and their answers.

    Args:
        questions: List of questions
        human_answers: List of ground truth answers
        predicted_answers: List of model-generated answers
        retrieved_contexts: Optional list of retrieved contexts (defaults to predicted answers if None)

    Returns:
        Dict containing evaluation results for each question and aggregate metrics
    """
    if len(questions) != len(human_answers) or len(questions) != len(predicted_answers):
        raise ValueError("All input lists must have the same length")

    if retrieved_contexts is None:
        retrieved_contexts = predicted_answers

    evaluator = RAGEvaluator()
    results = {}
    aggregate_scores = {metric: [] for metric in
                       ['context_relevance', 'answer_similarity_cosine', 'answer_similarity_bleu',
                        'answer_rouge1', 'answer_rouge2', 'answer_rougeL', 'faithfulness', 'overall_score']}

    for i, (question, context, pred_answer, human_answer) in enumerate(
        zip(questions, retrieved_contexts, predicted_answers, human_answers), 1):

        scores = evaluator.evaluate_single_response(
            question, context, pred_answer, human_answer)

        results[f'question_{i}'] = {
            'question': question,
            'human_answer': human_answer,
            'predicted_answer': pred_answer,
            'retrieved_context': context,
            'metrics': scores
        }

        for metric, score in scores.items():
            aggregate_scores[metric].append(score)

    # Add aggregate metrics
    results['aggregate'] = {
        metric: mean(scores) for metric, scores in aggregate_scores.items()
    }

    return results

if __name__ == "__main__":
    # Example questions and answers (same as in original code)
    questions = [
        "What is Amazon's return policy for electronics?",
        "How long do I have to return clothing items on Amazon?",
        # ... rest of the questions
    ]

    human_answers = [
        "Amazon typically allows returns for electronics within 30 days of receipt. However, items must be in new condition with all original packaging and accessories.",
        "Clothing items purchased on Amazon can generally be returned within 30 days of receipt, often with free returns.",
        # ... rest of the human answers
    ]

    predicted_answers = [
        "Electronics can usually be returned within 30 days. Check the specific product's return policy for details.",
        "Most clothing items are eligible for return within 30 days, as long as they are unworn and have tags.",
        # ... rest of the predicted answers
    ]

    results = evaluate_questions(questions, human_answers, predicted_answers)

    # Print results
    print("\nEvaluation Results\n")
    for question_id, data in results.items():
        if question_id == 'aggregate':
            print("\n=== Aggregate Metrics ===")
            for metric, score in data.items():
                print(f"{metric:25}: {score:.4f}")
        else:
            print(f"\n=== {question_id} ===")
            print(f"Question: {data['question']}")
            print(f"Human Answer: {data['human_answer']}")
            print(f"Predicted Answer: {data['predicted_answer']}")
            print("\nMetrics:")
            for metric, score in data['metrics'].items():
                print(f"{metric:25}: {score:.4f}")


Evaluation Results


=== question_1 ===
Question: What is Amazon's return policy for electronics?
Human Answer: Amazon typically allows returns for electronics within 30 days of receipt. However, items must be in new condition with all original packaging and accessories.
Predicted Answer: Electronics can usually be returned within 30 days. Check the specific product's return policy for details.

Metrics:
context_relevance        : 0.4523
answer_similarity_cosine : 0.2417
answer_similarity_bleu   : 0.0000
answer_rouge1            : 0.3415
answer_rouge2            : 0.1026
answer_rougeL            : 0.1951
faithfulness             : 1.0000
overall_score            : 0.3333

=== question_2 ===
Question: How long do I have to return clothing items on Amazon?
Human Answer: Clothing items purchased on Amazon can generally be returned within 30 days of receipt, often with free returns.
Predicted Answer: Most clothing items are eligible for return within 30 days, as long as they are unworn an